# Numerics of Empirical Risk Minimisation - Adversarial Logistic Classification

In [34]:
import matplotlib as mpl
import numpy as np
import scipy as sp
from scipy import special

from matplotlib import pyplot as plt
from matplotlib import style
style.use('default')
from matplotlib import rc, rcParams
rcParams['font.size'] = 20
import mpmath
import adversarial_loss_gradient as skloss
from data_model import *

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Could we just use mpmath?

In [35]:
def logsig_mpmath(x):
  if x > 0:
    return mpmath.log(1 / (1 + mpmath.exp(-x)))
  else:
    return x - mpmath.log(mpmath.exp(x) + 1)
  
def logsig_naive(t):
    return mpmath.log(1 / (1 + mpmath.exp(-t)))

n = 12
nsteps = 100
exps = np.concatenate([np.linspace(n, 0, nsteps), [0], np.linspace(0, n, nsteps)])
xs = 2 ** exps
xs[:nsteps] *= -1
xs[nsteps] = 0

# compute logsig using logsig_mpmath and logsig_naive
logsig_mpmath_xs = np.zeros(len(xs))
logsig_naive_xs = np.zeros(len(xs))
for i, x in enumerate(xs):
    logsig_mpmath_xs[i] = logsig_mpmath(x)
    logsig_naive_xs[i] = logsig_naive(x)
    # convert both to numpy float64
    logsig_mpmath_xs[i] = np.float64(logsig_mpmath_xs[i])
    logsig_naive_xs[i] = np.float64(logsig_naive_xs[i])
    
# compute the average difference between the two
diff = np.abs(logsig_mpmath_xs - logsig_naive_xs)
avg_diff = np.mean(diff)
std_diff = np.std(diff)
print("Average difference between logsig_mpmath and logsig_naive: {}".format(avg_diff))
print("Standard deviation of difference between logsig_mpmath and logsig_naive: {}".format(std_diff))

Average difference between logsig_mpmath and logsig_naive: 2.0989291012813905e-17
Standard deviation of difference between logsig_mpmath and logsig_naive: 1.4792252545044533e-16


In principle, it is possible to just use mpmath. However, at the moment it is unclear to me how well matrix operations are supported.
Hence, we might depend on slow for-loops and moving to cython and doing mpmath there might fail as it still requires the GIL. So it is slow.
This means that me might be only able to resort to mpmath in order to check out cython code

## Evaluating Cython Code

In [36]:
def cython_eval(z,e,y):

    half = skloss.CyHalfBinomialLoss()
    loss_out = np.empty_like(y)
    gradient_out = np.empty_like(z)
    epsilon_gradient_out = np.empty_like(z)
    half.loss_gradient( y_true=y,
        raw_prediction=z,    
        adversarial_norm = e,
        loss_out=loss_out,
        gradient_out=gradient_out,
        epsilon_gradient_out = epsilon_gradient_out,
        n_threads=1,
    )    

    return loss_out, gradient_out, epsilon_gradient_out

def mp_cython_eval(z,e,y):
    loss_out = np.empty_like(y)
    gradient_out = np.empty_like(z)
    epsilon_gradient_out = np.empty_like(z)

    skloss.mp_loss_gradient(y_true=y,
        raw_prediction=z,    
        adversarial_norm = e,
        loss_out=loss_out,
        gradient_out=gradient_out,
        epsilon_gradient_out = epsilon_gradient_out
    )

    return loss_out, gradient_out, epsilon_gradient_out

## Loss

$\mathcal{L}=\sum_{\mu=1}^N-y_\mu \frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}+y_\mu \frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}+\left(1-y_\mu\right) \log \left(1+\exp \left(\frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}+\frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}\right)\right)+y_\mu \log \left(1+\exp \left(\frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}-\frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}\right)\right)$

Let $z_\mu = \frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}$
Let $e = \frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}$

Then
$\mathcal{L}=\sum_{\mu=1}^N-y_\mu z_\mu+y_\mu e+\left(1-y_\mu\right) \log \left(1+\exp \left(z_\mu+e\right)\right)+y_\mu \log \left(1+\exp \left(z_\mu-e\right)\right)$


In [37]:
# Define the arguments of interest
n = 12
nsteps = 50
exps = np.concatenate([np.linspace(n, 0, nsteps), [0], np.linspace(0, n, nsteps)])
zs = 2 ** exps
zs[:nsteps] *= -1
zs[nsteps] = 0
ys = np.array([0,1])
es = zs.copy()

# Create an array with all possible arguments z,e,y
Z, E, Y = np.meshgrid(zs, es, ys)
Z = Z.flatten()
E = E.flatten()
Y = Y.flatten()

Y = Y.astype(np.float64)


In [38]:
# Fix the mpmath precision
# mpmath.mp.dps = 10000 

# The mpmath loss function for the baseline
def mp_loss(z,e,y):
    return -y*z + y*e + (1-y)*mpmath.log(1+mpmath.exp(z+e)) + y*mpmath.log(1+mpmath.exp(z-e))

# our cython code
def cython_loss(z,e,y):
    loss_out, _, _ = cython_eval(z,e,y)
    return loss_out

def mp_cython_loss(z,e,y):
    loss_out, _, _ = mp_cython_eval(z,e,y)
    return loss_out


def log1pexp(x):
    out = np.zeros_like(x)
    idx0 = x <= -37
    out[idx0] = np.exp(x[idx0])
    idx1 = (x > -37) & (x <= -2)
    out[idx1] = np.log1p(np.exp(x[idx1]))
    idx2 = (x > -2) & (x <= 18)
    out[idx2] = np.log(1. + np.exp(x[idx2]))
    idx3 = (x > 18) & (x <= 33.3)
    out[idx3] = x[idx3] + np.exp(-x[idx3])
    idx4 = x > 33.3
    out[idx4] = x[idx4]
    return out


def stable_loss(z,e,y):
    return -y*z + y*e + (1-y)*log1pexp(z+e) + y*log1pexp(z-e)


def naive_loss(z,e,y):
    return -y*z + y*e + (1-y)*np.log(1+np.exp(z+e)) + y*np.log(1+np.exp(z-e))


In [39]:
loss_mp = np.array([mp_loss(z,e,y) for z,e,y in zip(Z,E,Y)])
loss_mp = np.array([np.float64(x) for x in loss_mp])

In [40]:
loss_cython_mp = np.array([mp_cython_loss(np.array([z]),e,np.array([y])) for z,e,y in zip(Z,E,Y)])
loss_cython_mp = loss_cython_mp.flatten()

In [41]:
loss_cython = np.array([cython_loss(np.array([z]),e,np.array([y])) for z,e,y in zip(Z,E,Y)])
loss_cython = loss_cython.flatten()


In [42]:
loss_stable = np.array([stable_loss(z,e,y) for z,e,y in zip(Z,E,Y)])

In [43]:
loss_naive = np.array([naive_loss(z,e,y) for z,e,y in zip(Z,E,Y)])

/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/3876734477.py:38: RuntimeWarning: overflow encountered in exp
  return -y*z + y*e + (1-y)*np.log(1+np.exp(z+e)) + y*np.log(1+np.exp(z-e))
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/3876734477.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return -y*z + y*e + (1-y)*np.log(1+np.exp(z+e)) + y*np.log(1+np.exp(z-e))


In [44]:
# Compute the absolute and relative errors
abs_error = np.abs(loss_mp - loss_cython)


# Compute the average and standard deviation of the relative error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

# Print the results
print("Comparison between MP and Cython")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

Comparison between MP and Cython
Average absolute error: nan
Standard deviation of the absolute error: nan


In [45]:
# Compute the absolute and relative errors
abs_error = np.abs(loss_mp - loss_cython_mp)



# Compute the average and standard deviation of the relative error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

# Print the results
print("Comparison between MP and Cython MP")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

Comparison between MP and Cython MP
Average absolute error: 0.00e+00
Standard deviation of the absolute error: 0.00e+00


In [46]:
# Compute the absolute and relative errors
abs_error = np.abs(loss_mp - loss_stable)


# Compute the average and standard deviation of the relative error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

# Print the results
print("Comparison between MP and Stable")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

Comparison between MP and Stable
Average absolute error: 4.99e-18
Standard deviation of the absolute error: 7.99e-17


In [47]:
# Compute the absolute errors
abs_error = np.abs(loss_mp - loss_naive)

# Compute the average and standard deviation of the relative error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

# Print the results
print("Comparison between MP and Naive")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

Comparison between MP and Naive
Average absolute error: nan
Standard deviation of the absolute error: nan


In [48]:
arg_zip = np.array([[np.array([z]),e,np.array([y])] for z,e,y in zip(Z,E,Y)])

/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/2602079403.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arg_zip = np.array([[np.array([z]),e,np.array([y])] for z,e,y in zip(Z,E,Y)])


In [49]:
arg_zip[2]

array([array([-3456.51396913]), -4096.0, array([0.])], dtype=object)

In [50]:
cython_loss(*arg_zip[2])

array([0.])

In [51]:
loss_mp

array([0.00000000e+00, 6.93147181e-01, 0.00000000e+00, ...,
       6.39486031e+02, 8.19200000e+03, 6.93147181e-01])

In [52]:
loss_cython_mp

array([0.00000000e+00, 6.93147181e-01, 0.00000000e+00, ...,
       6.39486031e+02, 8.19200000e+03, 6.93147181e-01])

In [53]:
loss_stable

array([0.00000000e+00, 6.93147181e-01, 0.00000000e+00, ...,
       6.39486031e+02, 8.19200000e+03, 6.93147181e-01])

In [54]:
loss_cython

array([0.        , 0.69314718, 0.        , ...,       -inf,        nan,
             -inf])

## Gradient

$\begin{aligned} \partial_w \mathcal{L} & =\sum_{\mu=1}^N-y_\mu \frac{X_\mu}{\sqrt{d}}+y_\mu H+\frac{\left(1-y_\mu\right)}{1+\exp (-C)}\left(\frac{X_\mu}{\sqrt{d}}+H\right)+\frac{y_\mu}{1+\exp (-\bar{C})}\left(\frac{X_\mu}{\sqrt{d}}-H\right) \\ & =\sum_{\mu=1}^N\left[H \cdot\left(\frac{\left(1-y_\mu\right)}{1+\exp (-C)}+\frac{y_\mu \exp (-\bar{C})}{1+\exp (-\bar{C})}\right)+\frac{X_\mu}{\sqrt{d}} \cdot\left(\frac{\left(1-y_\mu\right)}{1+\exp (-C)}-\frac{y_\mu \exp (-\bar{C})}{1+\exp (-\bar{C})}\right)\right]\end{aligned}$

We are only interested in evaluating the function that will be multiplied with the data $X_\mu$ and the derivative of the optimal attack $H$.

Note the definitions
$C=\frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}+\frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}$ and $\bar{C}=\frac{X_\mu^T \boldsymbol{w}}{\sqrt{d}}-\frac{\varepsilon_t \boldsymbol{w}^T \boldsymbol{\Sigma}_\delta \boldsymbol{w}}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}}$

### Focusing on Sigmoids

From the gradient expression, it looks like it should be sufficient to compute the sigmoids accurately

In [55]:
def naive_sigmoid(x):
    return 1 / (1 + np.exp(-x))

def stable_sigmoid(x):
    out = np.zeros_like(x)
    idx = x <= 0
    out[idx] = np.exp(x[idx]) / (1 + np.exp(x[idx]))
    idx = x > 0
    out[idx] = 1 / (1 + np.exp(-x[idx]))
    return out

def mpmath_sigmoid(x):
    # return 1 / (1 + mpmath.exp(-x))
    if x <= 0:
        return mpmath.exp(x) / (1 + mpmath.exp(x))
    else:
        return 1 / (1 + mpmath.exp(-x))

naives = np.array([naive_sigmoid(x) for x in xs])
mpmaths = np.array([mpmath_sigmoid(x) for x in xs])
stables = np.array([stable_sigmoid(x) for x in xs])

# compute the abs difference between the two
diff = np.abs(naives - mpmaths)
avg_diff = np.mean(diff)
std_diff = np.std(diff)
print("Average difference between naive sigmoid and mpmath sigmoid: {}".format(avg_diff))
print("Standard deviation of difference between naive sigmoid and mpmath sigmoid: {}".format(std_diff))

# compute the abs difference between the two
diff = np.abs(mpmaths - stables)
avg_diff = np.mean(diff)
std_diff = np.std(diff)
print("Average difference between mpmath sigmoid and stable sigmoid: {}".format(avg_diff))
print("Standard deviation of difference between mpmath sigmoid and stable sigmoid: {}".format(std_diff))

# count infs and nans in the naive sigmoid
naive_infs = np.isinf(naives)
naive_nans = np.isnan(naives)
naive_inf_count = np.sum(naive_infs)
naive_nan_count = np.sum(naive_nans)
print("Naive sigmoid has {} infs and {} nans".format(naive_inf_count, naive_nan_count))

# count infs and nans in the stable sigmoid
stable_infs = np.isinf(stables)
stable_nans = np.isnan(stables)
stable_inf_count = np.sum(stable_infs)
stable_nan_count = np.sum(stable_nans)
print("Stable sigmoid has {} infs and {} nans".format(stable_inf_count, stable_nan_count))


Average difference between naive sigmoid and mpmath sigmoid: 1.24186408533529e-18
Standard deviation of difference between naive sigmoid and mpmath sigmoid: 6.01214443155874e-18
Average difference between mpmath sigmoid and stable sigmoid: 1.90530007013938e-334
Standard deviation of difference between mpmath sigmoid and stable sigmoid: 2.69450119958153e-333
Naive sigmoid has 0 infs and 0 nans
Stable sigmoid has 0 infs and 0 nans


/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/475322496.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


### Back to the gradient per sample

In [56]:

def mp_gradient(z,e,y):
    opt_attack_term = (1-y)*mpmath_sigmoid(z+e) + y*mpmath_sigmoid(-z+e)
    data_term = (1-y)*mpmath_sigmoid(z+e) - y*mpmath_sigmoid(-z+e)
    return opt_attack_term, data_term


def cython_gradient(z,e,y):
    _, gradient_out, attack_term = cython_eval(z,e,y)
    return attack_term, gradient_out

def mp_cython_gradient(z,e,y):
    _, gradient_out, attack_term = mp_cython_eval(z,e,y)
    return attack_term, gradient_out

def stable_gradient(z,e,y):
    opt_attack_term = (1-y)*stable_sigmoid(z+e) + y*stable_sigmoid(-z+e)
    data_term = (1-y)*stable_sigmoid(z+e) - y*stable_sigmoid(-z+e)
    return opt_attack_term, data_term

In [57]:
gradient_mp = np.array([mp_gradient(z,e,y) for z,e,y in zip(Z,E,Y)])
gradient_mp = np.array([np.float64(x) for x in gradient_mp])
gradient_mp = gradient_mp.flatten()

In [58]:
gradient_cython = np.array([cython_gradient(np.array([z]),e,np.array([y])) for z,e,y in zip(Z,E,Y)])
gradient_cython = gradient_cython.flatten()

In [59]:
mp_gradient_cython = np.array([mp_cython_gradient(np.array([z]),e,np.array([y])) for z,e,y in zip(Z,E,Y)])
mp_gradient_cython = mp_gradient_cython.flatten()

In [60]:
gradient_stable = np.array([stable_gradient(z,e,y) for z,e,y in zip(Z,E,Y)])
gradient_stable = gradient_stable.flatten()

In [61]:
# Compute the absolute and errors between gradient_mp and all other gradients
abs_error = np.abs(gradient_mp - gradient_cython)
abs_error_mp = np.abs(gradient_mp - mp_gradient_cython)
abs_error_stable = np.abs(gradient_mp - gradient_stable)

# Compute the average and standard deviation of the relative error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

avg_abs_error_mp = np.mean(abs_error_mp)
std_abs_error_mp = np.std(abs_error_mp)

avg_abs_error_stable = np.mean(abs_error_stable)
std_abs_error_stable = np.std(abs_error_stable)

# Print the results
print("Comparison between MP and Cython")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

print("Comparison between MP and Cython MP")
print("Average absolute error: %.2e" % avg_abs_error_mp)
print("Standard deviation of the absolute error: %.2e" % std_abs_error_mp)

print("Comparison between MP and Stable")
print("Average absolute error: %.2e" % avg_abs_error_stable)
print("Standard deviation of the absolute error: %.2e" % std_abs_error_stable)

Comparison between MP and Cython
Average absolute error: 6.64e-25
Standard deviation of the absolute error: 9.49e-23
Comparison between MP and Cython MP
Average absolute error: 1.91e-19
Standard deviation of the absolute error: 2.64e-18
Comparison between MP and Stable
Average absolute error: 6.64e-25
Standard deviation of the absolute error: 9.49e-23


In [62]:
# Count the number of nan and inf values in the gradient_cython
cython_infs = np.isinf(gradient_cython)
cython_nans = np.isnan(gradient_cython)
cython_inf_count = np.sum(cython_infs)
cython_nan_count = np.sum(cython_nans)
print("Cython gradient has {} infs and {} nans".format(cython_inf_count, cython_nan_count))

Cython gradient has 0 infs and 0 nans


## Hessian

$$
\begin{aligned}
\partial_{\boldsymbol{w}^2} \mathcal{L} & =\sum_{\mu=1}^N y_\mu \partial_{\boldsymbol{w}} H+\frac{\left(1-y_\mu\right)}{1+\exp (-C)}\left(\partial_{\boldsymbol{w}} H\right)+\frac{y_\mu}{1+\exp (-\bar{C})}\left(-\partial_{\boldsymbol{w}} H\right) \\
& +\left(1-y_\mu\right)\left(\frac{X_\mu}{\sqrt{d}}+H\right) \partial_{\boldsymbol{w}} \frac{1}{1+\exp (-C)}+y_\mu\left(\frac{X_\mu}{\sqrt{d}}-H\right) \partial_{\boldsymbol{w}} \frac{1}{1+\exp (-\bar{C})}
\end{aligned}
$$
Let's compute the individual terms. For now, we simplify $\mathrm{H}$ a bit We let $H=\frac{\varepsilon_t}{\sqrt{d}} \frac{w}{\sqrt{\boldsymbol{w}^T \boldsymbol{w}}}$. Thus, the derivative is
$$
\begin{gathered}
\partial_{\boldsymbol{w}} H=\frac{\varepsilon_t}{\sqrt{d} \sqrt{\boldsymbol{w}^T \boldsymbol{w}}} \mathbb{I}-\frac{\varepsilon_t}{\sqrt{d}} \frac{\boldsymbol{w} \boldsymbol{w}^T}{\left(\boldsymbol{w}^T \boldsymbol{w}\right)^{3 / 2}} \\
\partial_{\boldsymbol{w}} C_\mu=X_\mu^T+\frac{1}{2} \varepsilon_t \frac{\boldsymbol{w}^T}{\sqrt{\boldsymbol{w}^T \boldsymbol{w}}} \quad \partial_{\boldsymbol{w}} \bar{C}_\mu=X_\mu^T-\frac{1}{2} \varepsilon_t \frac{\boldsymbol{w}^T}{\sqrt{\boldsymbol{w}^T \boldsymbol{w}}} \\
\partial_{\boldsymbol{w}} \frac{1}{1+\exp (-C)}=\frac{-\partial_{\boldsymbol{w}} C}{2 \cosh C+2}
\end{gathered}
$$

Here we have to make sure to compute the sigmoids correctly and the cosh terms. The sigmoids we know already how to numerically compute, how about the coshs?

### Cosh Terms

In [63]:
def mp_stable_cosh_plus(z,e,y):
    if z+e < 0:
        return mpmath.exp(z+e)/( 1 + mpmath.exp(2*z+2*e) + 2*mpmath.exp(z+e) )
    else:
        return mpmath.exp( -z-e )/( 1 + mpmath.exp(-2*z-2*e) + 2*mpmath.exp(-z-e) )
    
def mp_stable_cosh_minus(z,e,y):
    if z-e < 0:
        return mpmath.exp(z-e)/( 1 + mpmath.exp(2*z-2*e) + 2*mpmath.exp(z-e) )
    else:
        return mpmath.exp( -z+e )/( 1 + mpmath.exp(-2*z+2*e) + 2*mpmath.exp(-z+e) )

def mp_cosh_plus(z,e,y):
    return 1/(2*mpmath.cosh(z+e) + 2)

def mp_cosh_minus(z,e,y):
    return 1/(2*mpmath.cosh(z-e) + 2)

def naive_cosh_plus(z,e,y):
    return 1/(2*np.cosh(z+e) + 2)

def naive_cosh_minus(z,e,y):
    return 1/(2*np.cosh(z-e) + 2)

def stable_cosh_plus(z,e,y):
    if z+e < 0:
        return np.exp( z+e )/( 1 + np.exp(2*z+2*e) + 2*np.exp(z+e) )
    else:
        return np.exp( -z-e )/( 1 + np.exp(-2*z-2*e) + 2*np.exp(-z-e) )
    
def stable_cosh_minus(z,e,y):
    if z-e < 0:
        return np.exp( z-e )/( 1 + np.exp(2*z-2*e) + 2*np.exp(z-e) )
    else:
        return np.exp( -z+e )/( 1 + np.exp(-2*z+2*e) + 2*np.exp(-z+e) )
    
def stable_cosh(x):
    out = np.zeros_like(x)
    idx = x <= 0
    out[idx] = np.exp(x[idx]) / (1 + np.exp(2*x[idx]) + 2*np.exp(x[idx]))
    idx = x > 0
    out[idx] = np.exp(-x[idx]) / (1 + np.exp(-2*x[idx]) + 2*np.exp(-x[idx]))
    return out

In [64]:
# Let's evaluate all the functions
cosh_plus_stable_mp = np.array([mp_stable_cosh_plus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_plus_stable_mp = np.array([np.float64(x) for x in cosh_plus_stable_mp])
cosh_plus_stable_mp = cosh_plus_stable_mp.flatten()

cosh_minus_stable_mp = np.array([mp_stable_cosh_minus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_minus_stable_mp = np.array([np.float64(x) for x in cosh_minus_stable_mp])
cosh_minus_stable_mp = cosh_minus_stable_mp.flatten()

cosh_plus_mp = np.array([mp_cosh_plus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_plus_mp = np.array([np.float64(x) for x in cosh_plus_mp])
cosh_plus_mp = cosh_plus_mp.flatten()

cosh_minus_mp = np.array([mp_cosh_minus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_minus_mp = np.array([np.float64(x) for x in cosh_minus_mp])
cosh_minus_mp = cosh_minus_mp.flatten()

cosh_plus_stable = np.array([stable_cosh_plus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_plus_stable = cosh_plus_stable.flatten()

cosh_minus_stable = np.array([stable_cosh_minus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_minus_stable = cosh_minus_stable.flatten()

cosh_plus_naive = np.array([naive_cosh_plus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_plus_naive = cosh_plus_naive.flatten()

cosh_minus_naive = np.array([naive_cosh_minus(z,e,y) for z,e,y in zip(Z,E,Y)])
cosh_minus_naive = cosh_minus_naive.flatten()

/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/4029768107.py:20: RuntimeWarning: overflow encountered in cosh
  return 1/(2*np.cosh(z+e) + 2)
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_85351/4029768107.py:23: RuntimeWarning: overflow encountered in cosh
  return 1/(2*np.cosh(z-e) + 2)


In [65]:
# compte the abs error between cosh_plus_stable_mp and cosh_plus_stable, cosh_plus_naive
abs_error = np.abs(cosh_plus_stable_mp - cosh_plus_stable)
abs_error_naive = np.abs(cosh_plus_stable_mp - cosh_plus_naive)
abs_error_mp = np.abs(cosh_plus_stable_mp - cosh_plus_mp)


# compute the average and standard deviation of the abs error
avg_abs_error = np.mean(abs_error)
std_abs_error = np.std(abs_error)

avg_abs_error_naive = np.mean(abs_error_naive)
std_abs_error_naive = np.std(abs_error_naive)

avg_abs_error_mp = np.mean(abs_error_mp)
std_abs_error_mp = np.std(abs_error_mp)

# print the results
print("Comparison between Stable MP and Stable")
print("Average absolute error: %.2e" % avg_abs_error)
print("Standard deviation of the absolute error: %.2e" % std_abs_error)

print("Comparison between Stable MP and Naive")
print("Average absolute error: %.2e" % avg_abs_error_naive)
print("Standard deviation of the absolute error: %.2e" % std_abs_error_naive)

print("Comparison between Stable MP and MP")
print("Average absolute error: %.2e" % avg_abs_error_mp)
print("Standard deviation of the absolute error: %.2e" % std_abs_error_mp)

Comparison between Stable MP and Stable
Average absolute error: 2.66e-24
Standard deviation of the absolute error: 1.90e-22
Comparison between Stable MP and Naive
Average absolute error: 3.82e-19
Standard deviation of the absolute error: 3.33e-18
Comparison between Stable MP and MP
Average absolute error: 3.82e-19
Standard deviation of the absolute error: 3.33e-18


### Back to the full Hessian

In [66]:
# The datamodel needs a logger
import logging
logger = logging.getLogger()
# Make the logger log to console
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)
# data_model = VanillaGaussianDataModel(1000,logger,source_pickle_path="",delete_existing=True)
# data_model = SourceCapacityDataModel(1000,logger,source_pickle_path="",delete_existing=True)
data_model = RandomCovariateDataModel(1000,logger,source_pickle_path="",delete_existing=True)
X,y,X_test, y_test, theta = data_model.generate_data(100, 0)

Assumption on Schur Complement failed: Matrix was not positive semi-definite; min eigval: -2.473374336499646e-11
Assumption on Schur Complement failed: Matrix was not positive semi-definite; min eigval: -2.473374336499646e-11
stored self to pickle
stored self to pickle


In [67]:
def compute_hessian(X,y,theta,epsilon):
    X = X / np.sqrt(X.shape[1])
    raw_prediction = X.dot(theta)

    # B - Optimal Attack ()
    B = epsilon * np.linalg.norm(theta) / np.sqrt(X.shape[1])

    # C and C_prime (n,)
    C = raw_prediction + B
    C_prime = raw_prediction - B

    # H - Derivative of Optimal Attack (d,)
    H = epsilon * theta / (np.linalg.norm(theta) * np.sqrt(X.shape[1]))

    # dH - Hessian of Optimal Attack (d,d)
    dH = np.eye(X.shape[1]) * epsilon / (np.linalg.norm(theta) * np.sqrt(X.shape[1])) - epsilon*np.outer(theta, theta) / (np.linalg.norm(theta) ** 3 * np.sqrt(X.shape[1]))

    # dH term
    vec = (1-y) * stable_sigmoid(C) + y * stable_sigmoid(-C_prime) # (n,)
    hessian = vec.sum() * dH

    # dC term and dC_prime term
    vecC = (1-y) * stable_cosh(C) # (n,)
    vecC_prime = y * stable_cosh(C_prime) # (n,)

    # Shift X by H
    X_plus = X + H
    X_minus = X - H

    # dC term
    act = np.multiply(X_plus.T, vecC) # (d,n)
    hessian += np.einsum('ij,ik->jk', X_plus, act.T) # (d,d)

    # dC_prime term
    act = np.multiply(X_minus.T, vecC_prime) # (d,n)
    hessian += np.einsum('ij,ik->jk', X_minus, act.T) # (d,d)

    return hessian



def logistic_hessian(X,y,theta):
    d = X.shape[1]
    raw_activation = X.dot(theta) / np.sqrt(d)
    B = np.multiply(X.T/np.sqrt(d),stable_cosh(raw_activation)) 
    a = np.einsum('ij,ik->jk', X/np.sqrt(d), B.T)
    assert a.shape == (d,d)
    return a


In [68]:
x1 = np.arange(9.0).reshape((3, 3))
x1

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [69]:
np.outer(x1[0],x1[0]) + np.outer(x1[1],x1[1]) + np.outer(x1[2],x1[2])

array([[45., 54., 63.],
       [54., 66., 78.],
       [63., 78., 93.]])

In [70]:
np.einsum("ij,ik->jk",x1,x1)

array([[45., 54., 63.],
       [54., 66., 78.],
       [63., 78., 93.]])

In [71]:
hessian_eps_0 = compute_hessian(X,y,theta,0.0)

In [72]:
log_hessian = logistic_hessian(X,y,theta)

In [73]:
hessian_eps_0_5 = compute_hessian(X,y,theta,0.5)

In [74]:
# Compute the hessians eigenvalues
eigvals_eps_0, _ = np.linalg.eig(hessian_eps_0)
eigvals_eps_0_5, _ = np.linalg.eig(hessian_eps_0_5)
eigvals_log_hess,_ = np.linalg.eig(log_hessian)

# print min max
print("Epsilon = 0")
print("Min eigenvalue: {}".format(np.min(eigvals_eps_0)))
print("Max eigenvalue: {}".format(np.max(eigvals_eps_0)))

print("Epsilon = 0.5")
print("Min eigenvalue: {}".format(np.min(eigvals_eps_0_5)))
print("Max eigenvalue: {}".format(np.max(eigvals_eps_0_5)))

print("Logistic Hessian")
print("Min eigenvalue: {}".format(np.min(eigvals_log_hess)))
print("Max eigenvalue: {}".format(np.max(eigvals_log_hess)))



Epsilon = 0
Min eigenvalue: (-1.5810887625069003e-16+0j)
Max eigenvalue: (0.9922374240901667+0j)
Epsilon = 0.5
Min eigenvalue: (-0.0009567625249476494+0j)
Max eigenvalue: (31.694422156878833+0j)
Logistic Hessian
Min eigenvalue: (-1.3637940013188962e-16-5.843069604875946e-17j)
Max eigenvalue: (0.9922374240902885+0j)
